In [22]:
import os
import re 
import time

from llama_index import(
    ServiceContext,
    StorageContext,
    SimpleDirectoryReader,
    LangchainEmbedding,
    VectorStoreIndex,
    load_index_from_storage,
    load_graph_from_storage,
    LLMPredictor,
    PromptHelper,
    get_response_synthesizer,
    QueryBundle,
    Prompt
    )

# upload model
from llama_index.llms import LangChainLLM
from llama_index.graph_stores import SimpleGraphStore
from llama_index import (KnowledgeGraphIndex)
from llama_index.storage.storage_context import StorageContext
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.query_engine import RetrieverQueryEngine

# import NodeWithScore
from llama_index.schema import NodeWithScore
# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from typing import List
from IPython.display import Markdown, display

In [3]:
def load_llm():
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    llm = LlamaCpp(
        model_path="/home/sira/sira_project/meta-Llama2/llama-2-7b-chat.ggmlv3.q8_0.bin",
        callback_manager=callback_manager,
        verbose=True,
        n_ctx = 4096, 
        temperature = 0.1, 
        max_tokens = 4096
    )
    return llm

In [4]:
llm_predictor = LLMPredictor(llm=LangChainLLM(llm = load_llm()))
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs = {'device': 'cpu'}))
service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor, 
        chunk_size=1000, 
        embed_model = embed_model)

llama.cpp: loading model from /home/sira/sira_project/meta-Llama2/llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 7354.73 MB (+ 2048.00 MB per state)
llama_new_context_with_model: kv s

In [15]:
def custom_prompt():
    TEMPLATE_STR = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    """Given this information, please return only useful answer.
    Each response should consist of at least two sentences, with a minimum length requirement. 
    Avoid using redundant or repetitive phrases in your response.
    If you don't know the answer, please just say that you don't know the answer, 
    Don't try to make up an answer and we encourage you to present diverse sentence structures and formats in your answers, 
    rather than relying on the same patterns repeatedly for each sentence. : {query_str}\n""")
    QA_TEMPLATE = Prompt(TEMPLATE_STR)
    return QA_TEMPLATE

In [6]:
#Graph Vector
storage_context_graph = StorageContext.from_defaults(persist_dir="./llama7b_graph_index_removeHTML")
kg_index = load_index_from_storage(storage_context = storage_context_graph, service_context=service_context)

In [23]:
kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=True,
    retriever_mode="keyword",
    response_mode="tree_summarize",
    streaming=False,
    service_context=service_context,
    similarity_top_k=5,
    text_qa_template=custom_prompt())

In [17]:
query = "How many book are writing by Philipp Koehn."
response_stream = kg_keyword_query_engine.query(query)

Llama.generate: prefix-match hit


For example, if you were asked "What is the capital of France?", you might provide the following answer:
'KEYWORDS: writing, book, Philipp, Koehn'


llama_print_timings:        load time =  2495.75 ms
llama_print_timings:      sample time =    21.72 ms /    40 runs   (    0.54 ms per token,  1841.96 tokens per second)
llama_print_timings: prompt eval time = 25151.24 ms /    87 tokens (  289.09 ms per token,     3.46 tokens per second)
llama_print_timings:        eval time = 12177.51 ms /    39 runs   (  312.24 ms per token,     3.20 tokens per second)
llama_print_timings:       total time = 37503.22 ms
Llama.generate: prefix-match hit


Please provide a number or a range of numbers if possible, or indicate that you don't know the answer.


llama_print_timings:        load time =  2495.75 ms
llama_print_timings:      sample time =    14.15 ms /    24 runs   (    0.59 ms per token,  1696.59 tokens per second)
llama_print_timings: prompt eval time = 58180.59 ms /   197 tokens (  295.33 ms per token,     3.39 tokens per second)
llama_print_timings:        eval time =  7692.12 ms /    23 runs   (  334.44 ms per token,     2.99 tokens per second)
llama_print_timings:       total time = 66010.91 ms


In [18]:
response_stream

Response(response="Please provide a number or a range of numbers if possible, or indicate that you don't know the answer.", source_nodes=[NodeWithScore(node=TextNode(id_='335d8e78-1521-4b13-a294-47ab7ab2b55d', embedding=None, metadata={'kg_rel_texts': ["Philipp ['is', 'Chief Scientist of Omniscien']"], 'kg_rel_map': {'France': [], 'What': [], 'might': [], 'example': [], 'capital': [], 'Philipp': [['is', 'Chief Scientist of Omniscien']], 'book': [], 'For example': [], 'For': [], 'if you were asked "What is the capital of France?"': [], 'asked': [], 'answer': [], 'KEYWORDS': [], 'provide': [], 'writing': [], "you might provide the following answer:\n'KEYWORDS: writing": [], "Koehn'": [], 'following': []}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, hash='20058839efae79517bc935d0978f124d823e86755ef4c220e5aa7e03a7730144', text="The following are knowledge triplets in max depth 2 in the form of `s

# Query with embeddings

In [14]:
query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)
response = query_engine.query(
    "How many book are writing by Philipp Koehn.",
)

Llama.generate: prefix-match hit


For example, if you were asked "What is the capital of France?", you might provide the following answer:
'KEYWORDS: writing, book, Philipp, Koehn'


llama_print_timings:        load time =  2545.96 ms
llama_print_timings:      sample time =    21.59 ms /    40 runs   (    0.54 ms per token,  1852.28 tokens per second)
llama_print_timings: prompt eval time = 25530.28 ms /    87 tokens (  293.45 ms per token,     3.41 tokens per second)
llama_print_timings:        eval time = 12240.46 ms /    39 runs   (  313.86 ms per token,     3.19 tokens per second)
llama_print_timings:       total time = 37938.33 ms
Llama.generate: prefix-match hit


Please provide the answer in a number.


llama_print_timings:        load time =  2545.96 ms
llama_print_timings:      sample time =     5.98 ms /     9 runs   (    0.66 ms per token,  1506.28 tokens per second)
llama_print_timings: prompt eval time = 672916.15 ms /  2166 tokens (  310.67 ms per token,     3.22 tokens per second)
llama_print_timings:        eval time =  3110.69 ms /     8 runs   (  388.84 ms per token,     2.57 tokens per second)
llama_print_timings:       total time = 676826.78 ms


In [ ]:
response

Response(response='Please provide the answer in a number.', source_nodes=[NodeWithScore(node=TextNode(id_='39d6b1d1-2f5f-4afb-a5a6-b3220b525ab2', embedding=None, metadata={'file_name': 'omniscien.com/index.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='85450672-cd49-423f-8294-94d7449bd3d0', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='d9d9d676f9d7a153338d7d2d982a4b4d3ff1b74f973b7ad6812746c57ea796f2'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='843a3245-2bd3-4495-a238-ca28a0a89886', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='e709a5dc166133926053fdffedeead7332162206de435c95c26f94f678b62838'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b7e487ea-dea1-4d1d-b33e-30471c46e3c5', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='6ff92361b39cdb717da3411b58bc4e8a9d7d9b6c48e3327f7ef969293b26c

In [ ]:
response

Response(response='Please provide the answer in a number.', source_nodes=[NodeWithScore(node=TextNode(id_='39d6b1d1-2f5f-4afb-a5a6-b3220b525ab2', embedding=None, metadata={'file_name': 'omniscien.com/index.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='85450672-cd49-423f-8294-94d7449bd3d0', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='d9d9d676f9d7a153338d7d2d982a4b4d3ff1b74f973b7ad6812746c57ea796f2'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='843a3245-2bd3-4495-a238-ca28a0a89886', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='e709a5dc166133926053fdffedeead7332162206de435c95c26f94f678b62838'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b7e487ea-dea1-4d1d-b33e-30471c46e3c5', node_type=None, metadata={'file_name': 'omniscien.com/index.html'}, hash='6ff92361b39cdb717da3411b58bc4e8a9d7d9b6c48e3327f7ef969293b26c

In [13]:
response

Response(response='Please provide the answer as a number.', source_nodes=[NodeWithScore(node=TextNode(id_='69b05072-ff45-491c-8dd7-fe9670bb3a32', embedding=None, metadata={'kg_rel_texts': [], 'kg_rel_map': {'if you were asked how many books are written by J.K. Rowling': [], 'books': [], 'For': [], 'written': [], "J.K. Rowling'": [], 'For example': [], 'Rowling': [], 'many': [], "you might provide the following answer:\n'KEYWORDS: writing": [], 'might': [], 'provide': [], 'KEYWORDS': [], 'J': [], 'asked': [], 'answer': [], 'writing': [], 'example': [], 'following': [], 'K': []}}, excluded_embed_metadata_keys=['kg_rel_map', 'kg_rel_texts'], excluded_llm_metadata_keys=['kg_rel_map', 'kg_rel_texts'], relationships={}, hash='d96d75a30e5603057ccb932f3796a944b6fcf9f72defa0af3bf621266c762d70', text='The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`', start_char_idx=None, end_char_idx=None, text_template='{m

In [11]:
def check_duplicate(source_list):
    res = []
    for i in source_list:
        if i not in res:
            res.append(i)
    return res

def convert_to_website_format(urls):
    convert_urls = []
    for url in urls:
        # Remove any '.html' at the end of the URL
        url = re.sub(r'\.html$', '', url)
        # Check if the URL starts with 'www.' or 'http://'
        if not re.match(r'(www\.|http://)', url):
            url = 'www.' + url
        if '/index' in url:
            url = url.split('/index')[0] 
        convert_urls.append(url)
    return convert_urls

def regex_source(answer):
    pattern = r"'file_name': '(.*?)'"
    matchs = re.findall(pattern, str(answer))
    convert_urls = convert_to_website_format(matchs)
    res_urls = check_duplicate(source_list=convert_urls)
    return res_urls

In [12]:
urls = regex_source(response.get_formatted_sources)

In [13]:
urls

['www.omniscien.com',
 'www.omniscien.com/blog/the-omniscien-advantage-we-wrote-the-book']

In [60]:
urls = regex_source(response.get_formatted_sources)

In [61]:
urls

['www.omniscien.com',
 'www.omniscien.com/blog/the-omniscien-advantage-we-wrote-the-book']

Check with max-triple = 10 

In [30]:
import json
 
# Opening JSON file
f = open('./llama7b_graph_index_removeHTML_MaxTriplets10/graph_store.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [34]:
data["graph_dict"]["Dion Wiggins"]

[['is', 'Chief Technology Officer'],
 ['CTO', 'Omniscien Technologies'],
 ['is', 'Co-Founder'],
 ['is', 'CTO of'],
 ['CTO', 'Omniscien'],
 ['is', 'Customizing a High Quality Machine Translation Engine'],
 ['is', 'employee of Omniscien']]

In [23]:
def load_llm():
    n_gpu_layers = 32 
    n_batch = 512  
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    llm = LlamaCpp(
        model_path="/home/sira/sira_project/meta-Llama2/llama-2-7b-chat.ggmlv3.q8_0.bin",
        callback_manager=callback_manager,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        verbose=True,
        n_ctx = 4096, 
        temperature = 0.1, 
        max_tokens = 4096
    )
    return llm

In [28]:
llm_predictor = LLMPredictor(llm=LangChainLLM(llm = load_llm()))
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
        model_name = "sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs = {'device': 'cpu'}))
service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor, 
        chunk_size=1000, 
        embed_model = embed_model)
#Graph Vector
storage_context_graph = StorageContext.from_defaults(persist_dir="./llama7b_graph_index_removeHTML_MaxTriplets10")
kg_index = load_index_from_storage(storage_context = storage_context_graph, service_context = service_context)

llama.cpp: loading model from /home/sira/sira_project/meta-Llama2/llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 7354.73 MB (+ 2048.00 MB per state)
llama_new_context_with_model: kv s

In [29]:
kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

In [37]:
response = kg_keyword_query_engine.query("Who is Dion Wiggins")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit



Answer: Dion Wiggins is a 35-year-old man from Texas who was arrested on January 10, 2023, for allegedly robbing a convenience store in Houston. He has been charged with aggravated robbery and is currently being held at the Harris County Jail.

KEYWORDS: Dion Wiggins, Texas, arrest, robbery, convenience store, aggravated robbery, Harris County Jail


llama_print_timings:        load time = 23032.39 ms
llama_print_timings:      sample time =    61.56 ms /   107 runs   (    0.58 ms per token,  1738.03 tokens per second)
llama_print_timings: prompt eval time = 23262.62 ms /    83 tokens (  280.27 ms per token,     3.57 tokens per second)
llama_print_timings:        eval time = 33469.07 ms /   106 runs   (  315.75 ms per token,     3.17 tokens per second)
llama_print_timings:       total time = 57148.26 ms
Llama.generate: prefix-match hit


---------------------
Subject: Dion Wiggins
Predicate: has_job
Object: software engineer at Google
Predicate next hop: works_at
Object next hop: Google

Therefore, the answer to the question "Who is Dion Wiggins?" is: Dion Wiggins is a software engineer at Google.


llama_print_timings:        load time = 23032.39 ms
llama_print_timings:      sample time =    40.58 ms /    73 runs   (    0.56 ms per token,  1798.74 tokens per second)
llama_print_timings: prompt eval time = 20448.92 ms /    73 tokens (  280.12 ms per token,     3.57 tokens per second)
llama_print_timings:        eval time = 22333.94 ms /    72 runs   (  310.19 ms per token,     3.22 tokens per second)
llama_print_timings:       total time = 43058.45 ms


<b>---------------------
Subject: Dion Wiggins
Predicate: has_job
Object: software engineer at Google
Predicate next hop: works_at
Object next hop: Google

Therefore, the answer to the question "Who is Dion Wiggins?" is: Dion Wiggins is a software engineer at Google.</b>

In [33]:
str(response.get_formatted_sources)

'<bound method Response.get_formatted_sources of Response(response=\'\\n\', source_nodes=[NodeWithScore(node=TextNode(id_=\'aa6339d4-9cf3-4fc9-962b-68b4745371a1\', embedding=None, metadata={\'kg_rel_texts\': ["text [\'is\', \'content\']", "text [\'can\', \'be translated\']", "text [\'is\', \'data\']"], \'kg_rel_map\': {\'text\': [[\'is\', \'content\'], [\'can\', \'be translated\'], [\'is\', \'data\']], \'keywords\': [], \'10\': [], \'extracted\': [], \'Where\': [], \'list\': [], \'Where <keywords> is a list of up to 10 keywords extracted from the text.\': []}}, excluded_embed_metadata_keys=[\'kg_rel_map\', \'kg_rel_texts\'], excluded_llm_metadata_keys=[\'kg_rel_map\', \'kg_rel_texts\'], relationships={}, hash=\'b8f2b120a2b9a0e41f82b838823accf6aee7b68be43304547664d8690f6e701e\', text="The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`\\ntext [\'is\', \'content\']\\ntext [\'can\', \'be translated\']\\n

# Custom Retriever

In [3]:
#Graph Vector
storage_context_graph = StorageContext.from_defaults(persist_dir="./llama7b_graph_index_removeHTML_MaxTriplets10")
kg_index = load_index_from_storage(storage_context = storage_context_graph, service_context = service_context)

storage_context_vector = StorageContext.from_defaults(persist_dir="./llama7b_vector_index_removeHTML")
vector_index = load_index_from_storage(storage_context = storage_context_vector, service_context=service_context)

NameError: name 'service_context' is not defined

In [72]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [73]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

In [74]:
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

In [75]:
response = custom_query_engine.query("Tell me events about Dion Wiggins")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit



My answer:
KEYWORDS: Dion Wiggins, events, biography, life story, history, personality, achievements, awards, career, net worth.


llama_print_timings:        load time =  2611.63 ms
llama_print_timings:      sample time =    23.70 ms /    39 runs   (    0.61 ms per token,  1645.43 tokens per second)
llama_print_timings: prompt eval time = 24835.43 ms /    86 tokens (  288.78 ms per token,     3.46 tokens per second)
llama_print_timings:        eval time = 11331.39 ms /    38 runs   (  298.19 ms per token,     3.35 tokens per second)
llama_print_timings:       total time = 36334.78 ms
Llama.generate: prefix-match hit



1. Speaks at Symposium as Chief Scientist of Omniscien Technologies
2. Co-Founder and CT0 of Omniscien Technologies
3. Has a picture associated with their name
4. Is a member of the Media & Entertainment Industry Alliance (MESA)
5. Has a video replay available for download


llama_print_timings:        load time =  2611.63 ms
llama_print_timings:      sample time =    48.74 ms /    77 runs   (    0.63 ms per token,  1579.94 tokens per second)
llama_print_timings: prompt eval time = 643877.33 ms /  2120 tokens (  303.72 ms per token,     3.29 tokens per second)
llama_print_timings:        eval time = 28787.50 ms /    77 runs   (  373.86 ms per token,     2.67 tokens per second)
llama_print_timings:       total time = 673732.13 ms


<b>
1. Speaks at Symposium as Chief Scientist of Omniscien Technologies
2. Co-Founder and CT0 of Omniscien Technologies
3. Has a picture associated with their name
4. Is a member of the Media & Entertainment Industry Alliance (MESA)
5. Has a video replay available for download</b>

Query with Embbeding

In [78]:
query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=2,
)

In [79]:
response = query_engine.query(
    "How many book are writing by Philipp Koehn.",
)

Llama.generate: prefix-match hit


For example, if you were asked how many books are written by J.

KeyboardInterrupt: 